# 排序算法

## 问题定义

**Input** 一个包含$n$个数字的序列 $s=<a_1, a_2, a_3, ..., a_n>$.

**Output**原序列$s$的其中一个排列(permutation)$s'=<a_1', a_2', a_3', ..., a_n'>$, 使得$a_1' \leq a_2' \leq ... \leq a_n'$.

---

## Heap Sort 堆排序

时间复杂度: $O(n log n)$

借助了数据结构 "堆 (Heap)", 所以在将堆排序之前我们需要先了解**堆**的一些基本性质.

### 堆

堆(二叉堆 binary heap) 是一种用**近似**完全二叉树来表示的数据结构 (*完全二叉树的意思为除了最底层, 其余层所有节点都有左右两个子节点*).

而堆也可以用数组来存储, 其中堆的根节点为Heap[1]. 

![堆的表示](http://users.cecs.anu.edu.au/~Alistair.Rendell/Teaching/apac_comp3600/module2/images/Heaps_HeapStructure.png)

对于堆中的某个节点$i$, 计算它的父节点和左右节点可以使用以下方法:

$parent(i)=floor({i \over 2})$

$left(i)=2i$

$right(i)=2i+1$

堆分为两种: **最大堆**和**最小堆**. 对于最大堆来说, 它的特性为:

任意节点$i \neq 1$, 有$Heap[parent(i)] \geq Heap[i]$. 所以最大堆中最大元素存放在根节点. 

最小堆与最大堆相反, 它的特性为:

任意节点$i \neq 1$, 有$Heap[parent(i)] \leq Heap[i]$. 所以最小堆中最小元素存放在根节点. 



In [1]:
def left(i):
    return i * 2

def right(i):
    return i * 2 + 1

def parent(i):
    return int(i / 2)

对于堆排序, 我们使用最大堆. 由二叉树的定义易知堆的$height=\Theta(log n)$

下面定义堆的几种操作:

### Max-heapify (Sink)

该操作用于维护最大堆的性质, 它的输入是一个数组$A$和一个下标$i$. 当它被调用时, 假设以$i$节点的两个子节点为根的两棵子树已经满足最大堆的性质, 而节点$i$的值A\[i\]有可能小于它子节点的值, 该情况不满足最大堆的定义. 所以在max-heapify操作中, A\[i\]会往下**下沉**, 完成操作后以$i$为根节点的子树将满足最大堆的性质. 我们将这个操作起名为下沉 (sink).

**举例** sink(a, 2)的过程模拟
![过程](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/heapify.png)

In [8]:
def sink(array, i, heap_size):
    l, r = left(i), right(i)
    largest = i
    if l <= heap_size and array[l] > array[largest]:
        largest = l
    if r <= heap_size and array[r] > array[largest]:
        largest = r
    if largest == i:
        return 
    array[i], array[largest] = array[largest], array[i]
    sink(array, largest, heap_size)

array = [0, 16, 4, 10, 14, 7, 9, 3, 2, 8, 1]
sink(array, 2, len(array) - 1)
print(array)

[0, 16, 14, 10, 8, 7, 9, 3, 2, 4, 1]


#### Sink操作的时间复杂度分析

1. 在判断节点$i$与其两个子节点的关系并且交换位置的步骤复杂度为$\Theta(1)$
2. 在递归调用某一个子节点时, 假设以$i$为根节点的树共有$n$个节点, 那么以其子节点为根的子树至多有$2n\over 3$个节点
3. 只有一次递归调用

综上, 我们可以写出Sink的$T(n)$如下:

$T(n) \leq T({2n \over 3}) + O(1)$

运用Master Theorem, 得出a=1, b=${3 \over 2}=1.5$, d=0. 

$$
T(n)=
\begin{cases}
O(n^dlogn)& ,\text{if } a=b^d\\
O(n^d)& ,\text{if } a < b^d\\
O(n^{log_ba})& ,\text{if } a > b^d
\end{cases}
$$

$b^d=1 = a$, 属于case 1, 所以复杂度为$O(logn)$

### Build heap

建堆的过程非常容易, 只需反复调用Sink操作即可实现. 通过分析我们知道假设数组$A$长度为$n$, 那么$A[n/2+1...n]$都是叶节点, 即它们都是长度为1的堆, 我们只需要关心前半部分也同样满足堆的性质即可.

In [10]:
def build_heap(array):
    heap_size = len(array) - 1
    for i in range(int(heap_size / 2), 0, -1):
        sink(array, i, heap_size)

array = [0, 6, 4, 10, 14, 7, 9, 3, 2, 8, 1]
build_heap(array)
print(array)

[0, 14, 8, 10, 6, 7, 9, 3, 2, 4, 1]


***思考*** 如何证明该操作的正确性?

#### Build heap的时间复杂度分析 \[重点\]

由前文可得到每一次Sink操作消耗$O(log n)$的时间, 并且build-heap进行了${n \over 2}=O(n)$次调用, 所以复杂度为$O(logn) \times O(n) = O(nlogn)$.

然而这是一个很松弛的渐进上界, 因为当对很底层的节点作sink操作时, 虽然堆中的节点数并没有发生改变, 但sink操作的时间会比对根节点进行sink操作短很多. 因此我们可以得出一个更好更紧密的上界. 

通过观察我们知道$n$个节点的堆的高度为$log n$, 且高度正好为$h$的节点数至多有${n \over 2^{h+1}}$个. 而高度为$h$的树, 对其根节点进行sink操作的复杂度为$O(h)$, 所以我们可以将build-heap的复杂度表示为
$$
O\Big(\sum_{h=0}^{logn} {n \over 2^{h+1}} O(h) \Big) = O\Big( n \sum_{h=0}^{logn} {h \over 2^{h+1}} \Big)
$$

令$k=logn, S=\sum_{h=0}^{k} {h \over 2^{h+1}}$, 则$2S=\sum_{h=0}^{k} {h \over 2^{h}}$.

$2S-S=({1\over2}+{2\over4}+{3\over8}+...+{k\over2^k}) - ({1\over4}+{2\over8}+...+{{i-1}\over2^i} + {i\over2^{i+1}})$

$S={1\over2} + {1\over4} + {1\over8} + ... + {1\over2^i} - {i\over2^{i+1}}$

显而易见, 
$$S \lt 1 - {i\over2^{i+1}} \lt 1$$

所以$O\Big( n \sum_{h=0}^{logn} {h \over 2^{h+1}} \Big)=O(n \times S)=O(n)$, 即build-heap的复杂度其实为$ O(n)$, 牛逼得抠脚!

### The MIGHTY heapsort algorithm

接下来就是堆排序的主体部分了. 用自然语言描述堆排序的过程为:

1. 对输入序列$A[1...n]$建堆 (build-heap操作).
2. 由于最大堆的定义是最大的值在根节点, 所以我们将$A[1]$与$A[n]$交换位置, 并将堆的大小-1.
3. 下沉$A[1]$, 因为此时$A[1]$是从最后直接上来的, 并不一定满足堆的性质.
4. 重复2, 3步骤, 直到堆中剩余元素为2.

经过上述过程之后的原序列$A[1...n]$即为排序后的序列, 也就是说, 堆排序是不额外消耗空间的.

图示过程:

![堆排序](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/heapsort.jpeg)

上代码:

In [31]:
def heapsort(array):
    # 在数组前添加一个占位符, 使得真正的数组从1开始
    array.insert(0, 0)
    
    # 建堆
    build_heap(array)
    heap_size = len(array) - 1
    
    # 从当前堆的最后一个叶节点开始与array[1]交换, 并将交换后的节点排除在堆之外
    # 最后下沉array[1]
    for i in range(len(array)-1, 1, -1):
        array[1], array[i] = array[i], array[1]
        heap_size -= 1
        sink(array, 1, heap_size)
    
    # 将最开始的占位符删掉
    array.pop(0)


a = [16, 1, 10, 8, 7, 9, 3, 2, 4, 14]
heapsort(a)
print(a)

[1, 2, 3, 4, 7, 8, 9, 10, 14, 16]


#### 时间复杂度分析

建堆需要$O(n)$的时间, 每次循环内的下沉需要$O(log n)$的时间, 循环n-1次, 所以复杂度为$O(n + n log n)=O(nlogn)$

---

## 快速排序

快速排序使用了分治的手段, 以下为使用分治排序序列$A[p...r]$的三个典型步骤:

### Divide

将$A[p...r]$分成两个子序列$A_1=A[p...q-1]$和$A_2=A[q+1...r]$, 使得$A_1$中所有元素都≤$A[q]$, 且$A_2$中所有元素都≥$A[q]$. 这一步骤的关键即为计算索引$q$, 该索引我们成为pivot. 

图示过程:

![partition](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/qsort_partition.png)

In [1]:
def partition(array, p, r):
    pivot = array[r]
    i = p - 1
    for j in range(p, r):
        if array[j] <= pivot:
            i += 1
            array[i], array[j] = array[j], array[i]
    array[i + 1], array[r] = array[r], array[i + 1]
    return i + 1

### Conquer

使用递归调用分别对$A_1, A_2$进行排序.

### Combine

将排序后的$A_1, A[q], A_2$拼接起来.


上述步骤的代码实现:

In [2]:
def quick_sort(array, p, r):
    if p < r:
        q = partition(array, p, r)
        quick_sort(array, p, q - 1)
        quick_sort(array, q + 1, r)

        
import random
a = [1, 4, 2, 9, 0, 6, 5, 7, 3, 10, 8, 1, 0]
quick_sort(a, 0, len(a) - 1)
print(a)

[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


### 效率

快速排序的运行效率取决于partition步骤是否能尽量平均地将原序列砍成两部分, 也就是说问题的关键落在了寻找合适的pivot上. 当partition很平均时, 快速排序的效率会非常高, 因为每一个子问题的规模都缩小了一半, 通过Master Theorem得知运行效率为$O(nlogn)$; 当partition不平均时, 其中一个子问题size为0, 另一个缩小为n-1, 这是非常糟糕的运行效率, 一开始我们就讲过.

#### 最坏情况直观分析

假设每次递归调用中我们都只能将序列分为n-1和0, 并且partition消耗$\Theta(n)$时间, 有:

$T(n)=T(n-1)+T(0)+\Theta(n) = T(n-1) + \Theta(n)=O(n^2)$ (如何证明?)

#### 最佳情况直观分析

若partition能做到最大限度的平均分, 那么得到的两个子问题规模都为原来的$1\over 2$, 这时快速排序会运行得非常快:

$T(n)=2T({n\over2})+\Theta(n)=O(nlogn)$ (如何证明?)

#### 一般情况直观分析

假设在每一次partition中我们分别得到两个子问题的规模为${n \over 10}$和${9n \over 10}$, 则:

$T(n)=T({9\over10}n)+T({n \over 10}) + cn$

根据定义, 在这里用cn来代替\Theta(n)是成立的. 接下来我们观察下图:

![recursion tree](https://ethanstorage.oss-cn-hangzhou.aliyuncs.com/quicksort_9_1split.png)

易知总是被分为$1\over10$的那一支的深度为$log_{10}n$, 因为问题规模每次都缩小为原来的$1 \over 10$. 类似地, 总是被分为$9\over10$的那一支深度为$log_{10\over9}n$. 而其他所有分支的深度都在这个范围之间. 且在当深度≤$log_{10}n$时, 这一层所有节点中的序列长度和为n, 即这一层中每个节点处理partition的复杂度之和为$\Theta(n)=cn$. 而再往下走, 每一层的partition复杂度之和一定会≤$cn$. 那么我们有如下关系:
$$
T(n)=O\big({log_{10}n \times cn} + (log_{10\over9}n - log_{10}n) \times cn \big)
=O\big( cnlogn + cn \times logn \big)=O(nlogn)
$$

通过以上分析我们得出结论, 就算是partition做得非常不好, 如9-1分, 甚至是99-1分, 只要能满足每次都是按照常数比例, 在渐进意义(asymptotically)上快速排序的时间复杂度总是为$O(nlogn)$.

### 随机化快速排序

很多算法都会采用随机化来获取更好的期望效率, 随机化同样可以用在快速排序中以获取更好的效率, 但做法并不是将原序列随机排序, 而是在partition过程中取代$pivot=A[r]$的做法, 让$pivot=A[i], i=random(p, r)$. 这种做法叫做**random sampling**. 

In [3]:
def randomized_partition(array, p, r):
    i = random.randint(p, r)
    array[i], array[r] = array[r], array[i]
    return partition(array, p, r)

使用随机化的partition来实现快速排序也非常容易, 用`randomized_partition`来取代原来的`partition`即可:

In [4]:
def randomized_quicksort(array, p, r):
    if p < r:
        q = randomized_partition(array, p, r)
        randomized_quicksort(array, p, q - 1)
        randomized_quicksort(array, q + 1, r)

        
a = [1, 4, 2, 9, 0, 6, 5, 7, 3, 10, 8, 1, 0]
randomized_quicksort(a, 0, len(a) - 1)
print(a)

[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


#### 复杂度分析

通过观察和分析算法, 不难知道快速排序的时间消耗主要集中在partition上. 每一次partition被调用时, 一个pivot会被选中, 并且在之后的所有递归调用中这个pivot将不再会被包含. 因此, 在整个快速排序的过程中partition至多被调用n次. 接下来分析partition中的时间消耗:
```python
1. def partition(array, p, r):
2.     pivot = array[r]
3.     i = p - 1
4.     for j in range(p, r):
5.         if array[j] <= pivot:
6.             i += 1
7.             array[i], array[j] = array[j], array[i]
8.     array[i + 1], array[r] = array[r], array[i + 1]
9.     return i + 1
```
在一次partition中, 2-3行消耗常数时间, 4-7行消耗时间取决于for循环的次数, 具体等于(循环次数)$\times$(5-7行消耗时间) (某个常数). 那么如果我们能算出在整个算法执行中有多少次第5行的比较, 就能得出有多少时间被消耗在了partition中的for循环上. 

设X为比较次数, 那么我们可以将快速排序的时间复杂度表示为$O(n+X)$. (想想如何证明这个表示是正确的)

**计算X**

要计算X, 我们必须理解数组中的两个元素在何时需要比较, 何时不需要. 我们假设有一个新的序列$Z=<z_1, z_2, ..., z_n>, z_i$表示数组A中第i小的元素. $Z_{ij}=<z_i, z_{i+1}, ..., z_j>$ 

在算法中, $z_i, z_j$至多会被比较一次. 因为partition中每个元素都只跟pivot作比较, 而pivot不会被第二次使用.

定义$X_{ij}=I\{z_i\text{是否与}z_j\text{比较}\}$

那么我们就有
$$
X=\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} X_{ij}
$$
同时对等号两边求数学期望:
$$
\begin{align*}
E[X]&=E\Big[   \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} X_{ij}       \Big] \\
    &=\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} E[X_{ij}] \\
    &=\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} p(z_i, z_j)
\end{align*}
$$
其中$p(a, b)$为a与b需要比较的概率. 算这个概率我们首先需要分析两个数不会比较的情况.

假设有一列数1到10, 我们选取了7作为pivot, 那么partition之后我们得到了两个序列, 分别用**集合**表示为{1, 2, 3, 4, 5, 6}与{8, 9, 10}. 在这个过程中, 7与所有数字都比较过一次, 但任意取这两个集合中的各一个元素, 二者都永远不会互相比较了. 

更客观地来描述, 假设我们从$Z_{ij}$中取出pivot$=x$, 且$z_i < x < z_j$, 那么$z_i$与$ z_j$将永不比较; 反之若$x \leq z_i$, 那么$z_i$将与$Z_{ij}$中的所有元素比较 (除了自己), 当$x \geq z_j$时也是类似的情况. 也就是说在$Z_{ij}中, $$z_i$和$z_j$需要比较的话, pivot就必须刚好选到它们二者之一. 由于我们是采取的随机选pivot的做法, 那么每个元素被选中的概率是均匀的, 在$Z_{ij}$中一共有$j-i+1$个元素, 所以每个元素被选中的概率是$1 \over {j-i+1}$, 即:
$$
\begin{align*}
p(z_i, z_j)&=p(z_i\text{被选中} | z_j\text{被选中})\\
           &=p(z_i\text{被选中}) + p(z_j\text{被选中}) \\
           &={1 \over j-i+1} \times 2 \\
           &={2 \over j-i+1}
\end{align*}
$$

代入原式
$$
E[X]=\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} {2 \over j-i+1}
$$
设$k=j-i$
$$
\begin{align*}
E[X]&=\sum_{i=1}^{n-1} \sum_{j=i+1}^{n} {2 \over j-i+1} \\
    &=\sum_{i=1}^{n-1} \sum_{k=1}^{n-i} {2 \over k + 1} \\
    &<\sum_{i=1}^{n-1} \sum_{k=1}^{n} {2 \over k} \\
\end{align*}
$$
求$\sum_{k=1}^n {2 \over k}$这个式子叫做**调和级数**, 很容易证明这个求和是发散的, 并没有通项公式, 那怎么办? 其实也很简单, 当k趋近于$+\infty$时, 它的和为:
$$
\lim_{n\to+\infty} \sum_{k=1}^n {2 \over k}=\int_{1}^{+\infty} {1 \over x} dx = ln(n)
$$
代入后有
$$
E[X]<\sum_{i=1}^{n-1} ln(n)=n \times ln(n)
$$
所以$O(nlogn)$